In [7]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex
)
from llama_index.core.node_parser import SentenceSplitter

In [9]:
import nest_asyncio

nest_asyncio.apply()

In [10]:
from settings import (
    COMPLETIONS_MODEL,
    API_EXCHANGE_VERSION,
    API_BASE_URL,
    EMBEDDINGS_BASE_URL,
    EMBEDDINGS_MODEL,
    TOKEN_ID
)

In [11]:
COMPLETIONS_MODEL

'gpt-35-turbo'

In [12]:
llm = AzureOpenAI(
    engine=COMPLETIONS_MODEL,
    api_key=TOKEN_ID,
    api_version=API_EXCHANGE_VERSION,
    azure_endpoint=f"{API_BASE_URL}/api"
)

embed_model = AzureOpenAIEmbedding(
    engine=EMBEDDINGS_MODEL,
    api_key=TOKEN_ID,
    api_version=API_EXCHANGE_VERSION,
    azure_endpoint=f"{EMBEDDINGS_BASE_URL}/api"
)

In [13]:
docs = SimpleDirectoryReader(input_files=["data/metagpt.pdf"]).load_data()

In [14]:
docs

[Document(id_='60ddd680-999e-4d1a-8eda-a8251540873a', embedding=None, metadata={'page_label': '1', 'file_name': 'metagpt.pdf', 'file_path': 'data\\metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-31', 'last_modified_date': '2024-05-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Preprint\nMETAGPT: M ETA PROGRAMMING FOR A\nMULTI -AGENT COLLABORATIVE FRAMEWORK\nSirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,\nCeyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,\nLiyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8\n1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,\n3Xiamen University,4The Chinese Univers

In [15]:
splitter = SentenceSplitter(chunk_size=1024)

nodes = splitter.get_nodes_from_documents(docs)

In [17]:
len(nodes)

34

In [18]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

### Defining Summary and Vector Indices

In [19]:
from llama_index.core import SummaryIndex


summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

### Defining QueryEngines and Set Metadata

In [20]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)

vector_query_engine = vector_index.as_query_engine()

In [21]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    )
)

vector_index_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from MetaGPT paper"
    )
)

### Defining a Router Query Engine

In [22]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_index_tool,
    ],
    verbose=True
)

In [30]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: The first choice mentions summarization questions related to MetaGPT, which directly relates to the question of summarizing the document..
Selecting query engine 1: The second choice mentions retrieving specific context from the MetaGPT paper, which is not directly related to summarizing the entire document..
The document introduces MetaGPT, a meta-programming framework for multi-agent collaboration based on Large Language Models (LLMs) that utilizes an assembly line paradigm to assign diverse roles to various agents, efficiently breaking down complex tasks into subtasks involving many agents working together. It incorporates Standardized Operating Procedures (SOPs) into prompt sequences for more streamlined workflows, allowing agents with human-like domain expertise to verify intermediate results and reduce errors. The document also discusses related work in automatic programming and LLM-based multi-agent frameworks, and presents a communication protocol and 

In [33]:
len(response.source_nodes)

36

In [34]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: The question is related to the specific content of the MetaGPT paper, and choice 2 is specifically relevant for retrieving context from the paper..
Agents in MetaGPT share information with other agents through a shared message pool. They publish their structured messages in the pool and can access messages from other entities transparently. Any agent can directly retrieve required information from the shared pool, eliminating the need to inquire about other agents and await their responses. Additionally, agents utilize role-specific interests to extract relevant information through a subscription mechanism.


In [35]:
response.response

'Agents in MetaGPT share information with other agents through a shared message pool. They publish their structured messages in the pool and can access messages from other entities transparently. Any agent can directly retrieve required information from the shared pool, eliminating the need to inquire about other agents and await their responses. Additionally, agents utilize role-specific interests to extract relevant information through a subscription mechanism.'

In [36]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: The ablation study results are specific context from the MetaGPT paper, which is mentioned in choice 2..
I'm sorry, but there is no information provided in the given context about any ablation study results.
